In [1]:
import torch
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torchvision.models import resnet50
from PIL import Image


In [ ]:
model = resnet50(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-2])
model.eval()

transform = transforms.Compose([
transforms.Resize((224, 224)),
transforms.ToTensor(),
transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dino_image = Image.open('/content/dino4.png').convert('RGB')
dino_tensor = transform(dino_image).unsqueeze(0)

with torch.no_grad():
  dinosaur_features = model(dino_tensor)


/home/i.zolotykh/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/i.zolotykh/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/i.zolotykh/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
  1%|▉                                                                             | 1.17M/97.8M [00:14<26:14, 64.4kB/s]

In [ ]:

model = resnet50(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-2])

all_image = Image.open('/content/drive/MyDrive/Colab Notebooks/dinosaurs/abc.jpg').convert('RGB')
all_tensor = transform(all_image).unsqueeze(0)

with torch.no_grad():
  all_features = model(all_tensor)

In [ ]:
print(dinosaur_features.shape)

In [ ]:
all_features = F.interpolate(all_features, size=(224, 224), mode='bilinear', align_corners=False)

In [ ]:
heatmap = F.conv2d(all_features, dinosaur_features)

heatmap = heatmap.squeeze().numpy()
heatmap = np.maximum(heatmap, 0)

plt.imshow(heatmap, cmap='hot')
plt.axis('off')
plt.colorbar(label='Feature Strength')
plt.show()